In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from catboost import CatBoostClassifier
from catboost import Pool

import os

In [9]:
def trainCatBoost(trainSet, targetSet, params={'depth': 7, 'rate': 0.03, 'l2': 8, 'T': 1},
                  folds=10, maxIter=1000, verbose=True, dirName =  '/tmp/porto/catboost/'):
    # create log directory
    dirName = dirName + str(params.get('depth')) + '_' + str(params.get('rate')) + \
              '_' + str(params.get('l2')) + '_' + str(params.get('T'))

    try:
        os.makedirs(dirName)
    except Exception as inst:
        print inst  # __str__ allows args to be printed directly

    treeList = list()
    scoreList = list()
    modelList = list()
    prob = np.zeros([trainSet.shape[0]])

    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

    for i, (train_index, val_index) in enumerate(skf.split(trainSet, targetSet)):
        model = CatBoostClassifier(verbose=verbose, iterations=maxIter, thread_count=4, eval_metric="AUC",
                                   depth=params['depth'], learning_rate=params['rate'],
                                   l2_leaf_reg=params['l2'], bagging_temperature=params['T'],
                                   od_type='Iter', od_wait=20,
                                   train_dir=dirName +"/"+ str(i), random_seed=i)

        # create pool
        trainPool = Pool(trainSet.iloc[train_index], targetSet.iloc[train_index],
                         feature_names=trainSet.columns.tolist())

        valPool = Pool(trainSet.iloc[val_index], targetSet.iloc[val_index],
                         feature_names=trainSet.columns.tolist())

        # fit and estimate the model
        model.fit(trainPool, eval_set=valPool, use_best_model=True)
        prob[val_index] = model.predict_proba(valPool)[:, 1]
        localScore = roc_auc_score(targetSet.iloc[val_index], prob[val_index])

        treeList.append(model.tree_count_)
        scoreList.append(localScore)
        modelList.append(model)
        
    score = roc_auc_score(targetSet, prob)
    return [score, treeList, scoreList, modelList]

In [10]:
 depth         int        7           
                l2            float      5.500000    
                rate          float      0.055000    
                T             float      1.500000    


SyntaxError: invalid syntax (<ipython-input-10-bccd38d145e2>, line 1)

In [11]:
params = {'depth': 7, 'l2': 5.5, 'rate': 0.055, 'T': 1.5}

fName = "/tmp/porto/data/train2.pcl"
df = pd.read_pickle(fName)
a = df.drop("target",axis = 1)
b = df.target

getResult = lambda x: trainCatBoost(a, b, x, maxIter=2000, folds=5, verbose= True)
[score, treeList, scoreList, modelList] = getResult(params)

Borders for float features generated
0:	learn 0.5706176509	test 0.5570739181	bestTest 0.5570739181		total: 540ms	remaining: 17m 58s
1:	learn 0.5776447509	test 0.5667472558	bestTest 0.5667472558		total: 1.09s	remaining: 18m 11s
2:	learn 0.5809440801	test 0.5708296706	bestTest 0.5708296706		total: 1.68s	remaining: 18m 35s
3:	learn 0.5927442505	test 0.5841645214	bestTest 0.5841645214		total: 2.25s	remaining: 18m 41s
4:	learn 0.5937256597	test 0.5855263631	bestTest 0.5855263631		total: 2.83s	remaining: 18m 51s
5:	learn 0.5984902891	test 0.5882513474	bestTest 0.5882513474		total: 3.44s	remaining: 19m 1s
6:	learn 0.6060764405	test 0.5926869132	bestTest 0.5926869132		total: 4.05s	remaining: 19m 13s
7:	learn 0.6126124766	test 0.6007632708	bestTest 0.6007632708		total: 4.68s	remaining: 19m 26s
8:	learn 0.6160093845	test 0.6033902043	bestTest 0.6033902043		total: 5.14s	remaining: 18m 57s
9:	learn 0.6179289591	test 0.605998495	bestTest 0.605998495		total: 5.74s	remaining: 19m 1s
10:	learn 0.61759

86:	learn 0.6431527851	test 0.6233318839	bestTest 0.6233318839		total: 49.8s	remaining: 18m 14s
87:	learn 0.6432891203	test 0.6233757074	bestTest 0.6233757074		total: 50.4s	remaining: 18m 14s
88:	learn 0.6435067074	test 0.6234636016	bestTest 0.6234636016		total: 51s	remaining: 18m 14s
89:	learn 0.6436506781	test 0.6234615522	bestTest 0.6234636016		total: 51.6s	remaining: 18m 15s
90:	learn 0.6439976526	test 0.6237043668	bestTest 0.6237043668		total: 52.3s	remaining: 18m 16s
91:	learn 0.6440859465	test 0.6238348729	bestTest 0.6238348729		total: 52.9s	remaining: 18m 17s
92:	learn 0.6442161043	test 0.62398254	bestTest 0.62398254		total: 53.5s	remaining: 18m 17s
93:	learn 0.6443302072	test 0.6240406211	bestTest 0.6240406211		total: 54.2s	remaining: 18m 18s
94:	learn 0.6444668379	test 0.6240426725	bestTest 0.6240426725		total: 54.8s	remaining: 18m 18s
95:	learn 0.6446849109	test 0.6240897752	bestTest 0.6240897752		total: 55.4s	remaining: 18m 19s
96:	learn 0.644888546	test 0.6243167188	bestTe

171:	learn 0.6550575338	test 0.6284572589	bestTest 0.6284660854		total: 1m 41s	remaining: 17m 59s
172:	learn 0.6551621101	test 0.6285397341	bestTest 0.6285397341		total: 1m 42s	remaining: 18m 1s
173:	learn 0.6553537078	test 0.6284901059	bestTest 0.6285397341		total: 1m 43s	remaining: 18m 1s
174:	learn 0.6554811604	test 0.6286093804	bestTest 0.6286093804		total: 1m 43s	remaining: 18m 2s
175:	learn 0.6556148927	test 0.6286818376	bestTest 0.6286818376		total: 1m 44s	remaining: 18m 3s
176:	learn 0.6557107309	test 0.6287138187	bestTest 0.6287138187		total: 1m 45s	remaining: 18m 4s
177:	learn 0.6557457506	test 0.6287590025	bestTest 0.6287590025		total: 1m 45s	remaining: 18m 4s
178:	learn 0.6558066649	test 0.6288100794	bestTest 0.6288100794		total: 1m 46s	remaining: 18m 5s
179:	learn 0.6559217478	test 0.6288234529	bestTest 0.6288234529		total: 1m 47s	remaining: 18m 6s
180:	learn 0.6559897094	test 0.6288385583	bestTest 0.6288385583		total: 1m 48s	remaining: 18m 6s
181:	learn 0.6560323116	test 

256:	learn 0.6639836604	test 0.6306627633	bestTest 0.6306627633		total: 2m 33s	remaining: 17m 21s
257:	learn 0.6641272257	test 0.6306844028	bestTest 0.6306844028		total: 2m 34s	remaining: 17m 20s
258:	learn 0.6642205234	test 0.6306338865	bestTest 0.6306844028		total: 2m 34s	remaining: 17m 19s
259:	learn 0.6643045706	test 0.6306677603	bestTest 0.6306844028		total: 2m 35s	remaining: 17m 19s
260:	learn 0.664352797	test 0.6306636514	bestTest 0.6306844028		total: 2m 35s	remaining: 17m 18s
261:	learn 0.6643820389	test 0.6306511157	bestTest 0.6306844028		total: 2m 36s	remaining: 17m 18s
262:	learn 0.664408997	test 0.6306643908	bestTest 0.6306844028		total: 2m 37s	remaining: 17m 18s
263:	learn 0.6644848588	test 0.6306175976	bestTest 0.6306844028		total: 2m 37s	remaining: 17m 17s
264:	learn 0.6645368018	test 0.6305781823	bestTest 0.6306844028		total: 2m 38s	remaining: 17m 17s
265:	learn 0.6646122438	test 0.6306001594	bestTest 0.6306844028		total: 2m 38s	remaining: 17m 16s
266:	learn 0.664754313

7:	learn 0.6122201254	test 0.6109697181	bestTest 0.6109697181		total: 4.39s	remaining: 18m 14s
8:	learn 0.6142839864	test 0.6120338869	bestTest 0.6120338869		total: 4.96s	remaining: 18m 16s
9:	learn 0.614694427	test 0.6122422501	bestTest 0.6122422501		total: 5.55s	remaining: 18m 24s
10:	learn 0.6153130956	test 0.6134120868	bestTest 0.6134120868		total: 5.98s	remaining: 18m 2s
11:	learn 0.6161552093	test 0.614753825	bestTest 0.614753825		total: 6.6s	remaining: 18m 13s
12:	learn 0.615231875	test 0.6138006778	bestTest 0.614753825		total: 7.24s	remaining: 18m 26s
13:	learn 0.6163986399	test 0.6150876592	bestTest 0.6150876592		total: 7.82s	remaining: 18m 29s
14:	learn 0.615699497	test 0.6141607798	bestTest 0.6150876592		total: 8.45s	remaining: 18m 37s
15:	learn 0.6163712363	test 0.6149905617	bestTest 0.6150876592		total: 9.07s	remaining: 18m 44s
16:	learn 0.6181070962	test 0.6159278611	bestTest 0.6159278611		total: 9.66s	remaining: 18m 47s
17:	learn 0.6203930231	test 0.618894572	bestTest 0.

...

4:	learn 0.5920624394	test 0.6038338708	bestTest 0.6038338708		total: 2.76s	remaining: 18m 20s
5:	learn 0.5959210822	test 0.6062864996	bestTest 0.6062864996		total: 3.35s	remaining: 18m 33s
6:	learn 0.6033899576	test 0.6122451402	bestTest 0.6122451402		total: 3.9s	remaining: 18m 29s
7:	learn 0.6031668284	test 0.6118858041	bestTest 0.6122451402		total: 4.37s	remaining: 18m 9s
8:	learn 0.6044802639	test 0.6121155765	bestTest 0.6122451402		total: 4.95s	remaining: 18m 14s
9:	learn 0.6043437331	test 0.6123666167	bestTest 0.6123666167		total: 5.53s	remaining: 18m 21s
10:	learn 0.6073067057	test 0.6133923731	bestTest 0.6133923731		total: 6.17s	remaining: 18m 34s
11:	learn 0.6082107894	test 0.6141118319	bestTest 0.6141118319		total: 6.86s	remaining: 18m 56s
12:	learn 0.609006559	test 0.6153324578	bestTest 0.6153324578		total: 7.47s	remaining: 19m 1s
13:	learn 0.6094323711	test 0.6162851518	bestTest 0.6162851518		total: 8.12s	remaining: 19m 12s
14:	learn 0.6092916163	test 0.6165495637	bestTest 

91:	learn 0.642218042	test 0.6368007108	bestTest 0.636803637		total: 54.3s	remaining: 18m 45s
92:	learn 0.6423441259	test 0.6369181871	bestTest 0.6369181871		total: 54.9s	remaining: 18m 45s
93:	learn 0.6425071201	test 0.6369610524	bestTest 0.6369610524		total: 55.6s	remaining: 18m 47s
94:	learn 0.6429314556	test 0.6372792434	bestTest 0.6372792434		total: 56.3s	remaining: 18m 49s
95:	learn 0.6430504852	test 0.6372700891	bestTest 0.6372792434		total: 57s	remaining: 18m 50s
96:	learn 0.6432540686	test 0.6373795445	bestTest 0.6373795445		total: 57.7s	remaining: 18m 51s
97:	learn 0.6434054401	test 0.6374243894	bestTest 0.6374243894		total: 58.3s	remaining: 18m 51s
98:	learn 0.6435135856	test 0.6373944828	bestTest 0.6374243894		total: 58.9s	remaining: 18m 51s
99:	learn 0.643818239	test 0.6375271344	bestTest 0.6375271344		total: 59.7s	remaining: 18m 53s
100:	learn 0.643959062	test 0.6374432989	bestTest 0.6375271344		total: 1m	remaining: 18m 54s
101:	learn 0.6440633016	test 0.6375483751	bestTe

176:	learn 0.6531935298	test 0.6404094023	bestTest 0.6404550028		total: 1m 49s	remaining: 18m 43s
177:	learn 0.6533582352	test 0.6404756507	bestTest 0.6404756507		total: 1m 49s	remaining: 18m 43s
178:	learn 0.6535439154	test 0.6405869128	bestTest 0.6405869128		total: 1m 50s	remaining: 18m 41s
179:	learn 0.653668877	test 0.6406272861	bestTest 0.6406272861		total: 1m 50s	remaining: 18m 41s
180:	learn 0.6538107288	test 0.6406583202	bestTest 0.6406583202		total: 1m 51s	remaining: 18m 40s
181:	learn 0.6539983598	test 0.6406804834	bestTest 0.6406804834		total: 1m 52s	remaining: 18m 40s
182:	learn 0.6542030994	test 0.64068721	bestTest 0.64068721		total: 1m 52s	remaining: 18m 40s
183:	learn 0.6544771591	test 0.6407829973	bestTest 0.6407829973		total: 1m 53s	remaining: 18m 39s
184:	learn 0.6545811051	test 0.6407675968	bestTest 0.6407829973		total: 1m 54s	remaining: 18m 38s
185:	learn 0.6547392597	test 0.6408719877	bestTest 0.6408719877		total: 1m 54s	remaining: 18m 37s
186:	learn 0.6548354445	t

261:	learn 0.6623945558	test 0.6422320721	bestTest 0.6422320721		total: 2m 39s	remaining: 17m 34s
262:	learn 0.6624705387	test 0.642262242	bestTest 0.642262242		total: 2m 39s	remaining: 17m 34s
263:	learn 0.6624852544	test 0.6422645311	bestTest 0.6422645311		total: 2m 40s	remaining: 17m 33s
264:	learn 0.6625425718	test 0.6422805345	bestTest 0.6422805345		total: 2m 40s	remaining: 17m 32s
265:	learn 0.6625719302	test 0.6422491487	bestTest 0.6422805345		total: 2m 41s	remaining: 17m 31s
266:	learn 0.6627180693	test 0.6422424302	bestTest 0.6422805345		total: 2m 41s	remaining: 17m 30s
267:	learn 0.6627631538	test 0.6422405531	bestTest 0.6422805345		total: 2m 42s	remaining: 17m 30s
268:	learn 0.6629209008	test 0.6422912142	bestTest 0.6422912142		total: 2m 43s	remaining: 17m 29s
269:	learn 0.6630968619	test 0.6423396143	bestTest 0.6423396143		total: 2m 43s	remaining: 17m 29s
270:	learn 0.6632058296	test 0.6423329802	bestTest 0.6423396143		total: 2m 44s	remaining: 17m 28s
271:	learn 0.663269557

346:	learn 0.6696581372	test 0.6426812647	bestTest 0.6428357942		total: 3m 29s	remaining: 16m 36s
347:	learn 0.6697252166	test 0.6426544289	bestTest 0.6428357942		total: 3m 29s	remaining: 16m 35s
348:	learn 0.6698661293	test 0.6426457671	bestTest 0.6428357942		total: 3m 30s	remaining: 16m 35s
349:	learn 0.6699547392	test 0.6427244235	bestTest 0.6428357942		total: 3m 31s	remaining: 16m 35s
350:	learn 0.6700521175	test 0.6427277737	bestTest 0.6428357942		total: 3m 31s	remaining: 16m 34s
351:	learn 0.6700993281	test 0.642792762	bestTest 0.6428357942		total: 3m 32s	remaining: 16m 33s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.6428357942
bestIteration = 330

Shrink model to first 331 iterations.


In [13]:
testDf = pd.read_pickle("/tmp/porto/data/test2.pcl")
testDf.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
id,,,,,,,,,,,,,,,,,,,,,
0,0,1,8,1,0,0,1,0,0,0,...,1,1,2,1,65,1,0.316228,0.669556,0.352136,3.464102
1,4,2,5,1,0,0,0,0,1,0,...,1,1,0,1,103,1,0.316228,0.606320,0.358329,2.828427
2,5,1,3,0,0,0,0,0,1,0,...,1,1,2,1,29,3,0.400000,0.896239,0.398497,3.316625
3,0,1,6,0,0,1,0,0,0,0,...,1,1,2,1,40,2,0.374166,0.652110,0.381445,2.449490
4,5,1,7,0,0,0,0,0,1,0,...,1,1,2,1,101,3,0.374166,0.812914,0.385097,3.316625


In [14]:
testPool = Pool(testDf, feature_names=testDf.columns.tolist())

In [16]:
2*score-1

0.28156419164553737

In [31]:
treeList

[311, 400, 321, 322, 331]

In [21]:
proba = np.ndarray([testDf.shape[0],5]) 
for i, model in enumerate(modelList):
    proba[:,i] = model.predict_proba(testPool)[:,1]

In [24]:
proba.mean(axis=1)

array([ 0.02455887,  0.02626326,  0.0256727 , ...,  0.03349493,
        0.02317673,  0.02941474])

In [28]:
def submit(pred, fName = 'submission.csv'):
    ## make submission file
    df = pd.DataFrame(pred,columns=['target'],index=testDf.index)
    df.to_csv(fName,index_label='id')

In [29]:
submit(proba.mean(axis=1),'submission11.csv')

In [34]:
proba = np.ndarray([testDf.shape[0],5]) 
for seed in range(0,5):

    model = CatBoostClassifier(verbose= True, iterations= 450, thread_count= 4, eval_metric= "AUC",
                               depth= params['depth'], learning_rate= params['rate'],
                               l2_leaf_reg= params['l2'], bagging_temperature= params['T'],
                               random_seed= seed)
    model.fit(df.drop('target',axis=1), df.target)
    proba[:,seed] = model.predict_proba(testPool)[:,1]
    
submit(proba.mean(axis=1),'submission12.csv')    

Borders for float features generated
0:	learn 0.5000934936	total: 1.32s	remaining: 9m 51s
1:	learn 0.5672959221	total: 2.07s	remaining: 7m 44s
2:	learn 0.584546325	total: 2.7s	remaining: 6m 42s
3:	learn 0.5940151299	total: 3.56s	remaining: 6m 36s
4:	learn 0.5971203595	total: 4.33s	remaining: 6m 25s
5:	learn 0.5999253437	total: 5.05s	remaining: 6m 13s
6:	learn 0.6065022303	total: 5.77s	remaining: 6m 5s
7:	learn 0.6056495869	total: 6.51s	remaining: 6m
8:	learn 0.6069375993	total: 7.48s	remaining: 6m 6s
9:	learn 0.6095271382	total: 8.24s	remaining: 6m 2s
10:	learn 0.6102613597	total: 8.92s	remaining: 5m 56s
11:	learn 0.6124226506	total: 9.57s	remaining: 5m 49s
12:	learn 0.6155577105	total: 10.1s	remaining: 5m 39s
13:	learn 0.6181297131	total: 11.8s	remaining: 6m 8s
14:	learn 0.61826191	total: 12.6s	remaining: 6m 4s
15:	learn 0.6206948568	total: 13.7s	remaining: 6m 11s
16:	learn 0.6216294446	total: 14.4s	remaining: 6m 6s
17:	learn 0.6217101168	total: 16.1s	remaining: 6m 25s
18:	learn 0.621

150:	learn 0.6486261008	total: 2m 15s	remaining: 4m 27s
151:	learn 0.648751842	total: 2m 16s	remaining: 4m 27s
152:	learn 0.6488272353	total: 2m 17s	remaining: 4m 25s
153:	learn 0.6488827557	total: 2m 17s	remaining: 4m 24s
154:	learn 0.6489581806	total: 2m 18s	remaining: 4m 23s
155:	learn 0.6491606731	total: 2m 20s	remaining: 4m 24s
156:	learn 0.6493036862	total: 2m 21s	remaining: 4m 23s
157:	learn 0.6494229425	total: 2m 22s	remaining: 4m 22s
158:	learn 0.6495846083	total: 2m 22s	remaining: 4m 21s
159:	learn 0.6496682639	total: 2m 23s	remaining: 4m 20s
160:	learn 0.6497186627	total: 2m 24s	remaining: 4m 18s
161:	learn 0.6498475302	total: 2m 24s	remaining: 4m 17s
162:	learn 0.6499170963	total: 2m 25s	remaining: 4m 16s
163:	learn 0.6499319463	total: 2m 26s	remaining: 4m 15s
164:	learn 0.6500113033	total: 2m 27s	remaining: 4m 14s
165:	learn 0.6500970842	total: 2m 28s	remaining: 4m 14s
166:	learn 0.6502062172	total: 2m 30s	remaining: 4m 15s
167:	learn 0.6502467013	total: 2m 31s	remaining: 

298:	learn 0.6615483077	total: 4m 36s	remaining: 2m 19s
299:	learn 0.6616243297	total: 4m 37s	remaining: 2m 18s
300:	learn 0.6617248708	total: 4m 39s	remaining: 2m 18s
301:	learn 0.6617393932	total: 4m 40s	remaining: 2m 17s
302:	learn 0.6618200026	total: 4m 41s	remaining: 2m 16s
303:	learn 0.6618831566	total: 4m 41s	remaining: 2m 15s
304:	learn 0.6619073996	total: 4m 42s	remaining: 2m 14s
305:	learn 0.6619973847	total: 4m 43s	remaining: 2m 13s
306:	learn 0.6620851615	total: 4m 44s	remaining: 2m 12s
307:	learn 0.6621286883	total: 4m 44s	remaining: 2m 11s
308:	learn 0.6621784788	total: 4m 46s	remaining: 2m 10s
309:	learn 0.6622671898	total: 4m 46s	remaining: 2m 9s
310:	learn 0.662319393	total: 4m 47s	remaining: 2m 8s
311:	learn 0.662418818	total: 4m 48s	remaining: 2m 7s
312:	learn 0.6624875106	total: 4m 49s	remaining: 2m 6s
313:	learn 0.66254871	total: 4m 50s	remaining: 2m 5s
314:	learn 0.6627021706	total: 4m 50s	remaining: 2m 4s
315:	learn 0.6627965901	total: 4m 51s	remaining: 2m 3s
316

447:	learn 0.6710030935	total: 6m 53s	remaining: 1.85s
448:	learn 0.6710237151	total: 6m 54s	remaining: 923ms
449:	learn 0.6711100189	total: 6m 55s	remaining: 0us
Borders for float features generated
0:	learn 0.565882635	total: 852ms	remaining: 6m 22s
1:	learn 0.5801185719	total: 1.55s	remaining: 5m 48s
2:	learn 0.5802060839	total: 1.94s	remaining: 4m 48s
3:	learn 0.5849787811	total: 2.67s	remaining: 4m 57s
4:	learn 0.5933050696	total: 3.36s	remaining: 4m 58s
5:	learn 0.5930679968	total: 4.82s	remaining: 5m 56s
6:	learn 0.6042746501	total: 5.52s	remaining: 5m 49s
7:	learn 0.6078211968	total: 6.23s	remaining: 5m 44s
8:	learn 0.6091700901	total: 8.37s	remaining: 6m 50s
9:	learn 0.6100054967	total: 9.81s	remaining: 7m 11s
10:	learn 0.6098413492	total: 10.4s	remaining: 6m 57s
11:	learn 0.6122821879	total: 11.2s	remaining: 6m 48s
12:	learn 0.6154900502	total: 12.5s	remaining: 6m 58s
13:	learn 0.6163956339	total: 13.1s	remaining: 6m 49s
14:	learn 0.6167153134	total: 14.3s	remaining: 6m 54s
1

148:	learn 0.6485447912	total: 2m 20s	remaining: 4m 43s
149:	learn 0.6487225623	total: 2m 21s	remaining: 4m 43s
150:	learn 0.6487554917	total: 2m 22s	remaining: 4m 41s
151:	learn 0.6487973415	total: 2m 22s	remaining: 4m 40s
152:	learn 0.6488731035	total: 2m 23s	remaining: 4m 38s
153:	learn 0.6490988843	total: 2m 24s	remaining: 4m 37s
154:	learn 0.6492147648	total: 2m 25s	remaining: 4m 36s
155:	learn 0.6492846625	total: 2m 25s	remaining: 4m 34s
156:	learn 0.6493678914	total: 2m 26s	remaining: 4m 33s
157:	learn 0.6494418855	total: 2m 27s	remaining: 4m 32s
158:	learn 0.6496389964	total: 2m 28s	remaining: 4m 31s
159:	learn 0.6497321884	total: 2m 30s	remaining: 4m 32s
160:	learn 0.6497649183	total: 2m 30s	remaining: 4m 30s
161:	learn 0.6498250095	total: 2m 31s	remaining: 4m 29s
162:	learn 0.6499525017	total: 2m 32s	remaining: 4m 28s
163:	learn 0.6501723903	total: 2m 34s	remaining: 4m 28s
164:	learn 0.6502305547	total: 2m 34s	remaining: 4m 27s
165:	learn 0.6502913727	total: 2m 35s	remaining:

...

292:	learn 0.6617600613	total: 4m 28s	remaining: 2m 23s
293:	learn 0.6617842724	total: 4m 29s	remaining: 2m 23s
294:	learn 0.6618789083	total: 4m 30s	remaining: 2m 22s
295:	learn 0.6619311222	total: 4m 31s	remaining: 2m 21s
296:	learn 0.6619959348	total: 4m 32s	remaining: 2m 20s
297:	learn 0.6620120118	total: 4m 32s	remaining: 2m 19s
298:	learn 0.6620833522	total: 4m 33s	remaining: 2m 18s
299:	learn 0.662182579	total: 4m 34s	remaining: 2m 17s
300:	learn 0.6622068822	total: 4m 35s	remaining: 2m 16s
301:	learn 0.6622600646	total: 4m 36s	remaining: 2m 15s
302:	learn 0.6623802145	total: 4m 36s	remaining: 2m 14s
303:	learn 0.6624062182	total: 4m 37s	remaining: 2m 13s
304:	learn 0.6624735309	total: 4m 38s	remaining: 2m 12s
305:	learn 0.6624917338	total: 4m 39s	remaining: 2m 11s
306:	learn 0.6625476675	total: 4m 39s	remaining: 2m 10s
307:	learn 0.6626292457	total: 4m 40s	remaining: 2m 9s
308:	learn 0.6626864009	total: 4m 41s	remaining: 2m 8s
309:	learn 0.662878953	total: 4m 41s	remaining: 2m 

441:	learn 0.6715042359	total: 6m 45s	remaining: 7.34s
442:	learn 0.6715085518	total: 6m 46s	remaining: 6.42s
443:	learn 0.6715488667	total: 6m 46s	remaining: 5.5s
444:	learn 0.6715853464	total: 6m 47s	remaining: 4.58s
445:	learn 0.6716323689	total: 6m 48s	remaining: 3.66s
446:	learn 0.6716823602	total: 6m 49s	remaining: 2.75s
447:	learn 0.6717218128	total: 6m 50s	remaining: 1.83s
448:	learn 0.6717684577	total: 6m 51s	remaining: 915ms
449:	learn 0.6718109054	total: 6m 51s	remaining: 0us
Borders for float features generated
0:	learn 0.5761797828	total: 2.57s	remaining: 19m 13s
1:	learn 0.5956832908	total: 4.37s	remaining: 16m 19s
2:	learn 0.5999237845	total: 5.02s	remaining: 12m 28s
3:	learn 0.6010781133	total: 5.72s	remaining: 10m 38s
4:	learn 0.6059915704	total: 6.46s	remaining: 9m 35s
5:	learn 0.6088876362	total: 7.17s	remaining: 8m 51s
6:	learn 0.6129046177	total: 7.96s	remaining: 8m 24s
7:	learn 0.6171366659	total: 9.34s	remaining: 8m 36s
8:	learn 0.6209919284	total: 10s	remaining:

142:	learn 0.6492966	total: 2m 17s	remaining: 4m 55s
143:	learn 0.6497389712	total: 2m 18s	remaining: 4m 54s
144:	learn 0.6498330276	total: 2m 19s	remaining: 4m 52s
145:	learn 0.6498813242	total: 2m 19s	remaining: 4m 51s
146:	learn 0.6499359915	total: 2m 20s	remaining: 4m 50s
147:	learn 0.649971393	total: 2m 21s	remaining: 4m 49s
148:	learn 0.6500928833	total: 2m 22s	remaining: 4m 48s
149:	learn 0.6501897945	total: 2m 23s	remaining: 4m 47s
150:	learn 0.6503453984	total: 2m 24s	remaining: 4m 45s
151:	learn 0.6504243113	total: 2m 25s	remaining: 4m 44s
152:	learn 0.6505406206	total: 2m 26s	remaining: 4m 45s
153:	learn 0.6506631841	total: 2m 27s	remaining: 4m 43s
154:	learn 0.6507763363	total: 2m 28s	remaining: 4m 42s
155:	learn 0.6510000296	total: 2m 29s	remaining: 4m 41s
156:	learn 0.651080015	total: 2m 30s	remaining: 4m 40s
157:	learn 0.6511414977	total: 2m 31s	remaining: 4m 39s
158:	learn 0.6512539258	total: 2m 32s	remaining: 4m 38s
159:	learn 0.6513906196	total: 2m 32s	remaining: 4m 3

290:	learn 0.6623621557	total: 4m 31s	remaining: 2m 28s
291:	learn 0.6624316139	total: 4m 31s	remaining: 2m 27s
292:	learn 0.6624768073	total: 4m 32s	remaining: 2m 26s
293:	learn 0.662562985	total: 4m 33s	remaining: 2m 25s
294:	learn 0.6626007418	total: 4m 34s	remaining: 2m 24s
295:	learn 0.6627184799	total: 4m 36s	remaining: 2m 23s
296:	learn 0.6628317203	total: 4m 36s	remaining: 2m 22s
297:	learn 0.6628516117	total: 4m 37s	remaining: 2m 21s
298:	learn 0.662918982	total: 4m 38s	remaining: 2m 20s
299:	learn 0.6629746108	total: 4m 39s	remaining: 2m 19s
300:	learn 0.6630268071	total: 4m 40s	remaining: 2m 18s
301:	learn 0.6631250151	total: 4m 41s	remaining: 2m 17s
302:	learn 0.663148318	total: 4m 42s	remaining: 2m 16s
303:	learn 0.6631701631	total: 4m 42s	remaining: 2m 15s
304:	learn 0.6632163002	total: 4m 43s	remaining: 2m 14s
305:	learn 0.6632226738	total: 4m 44s	remaining: 2m 13s
306:	learn 0.6632623215	total: 4m 45s	remaining: 2m 12s
307:	learn 0.6633218295	total: 4m 46s	remaining: 2m

439:	learn 0.672082484	total: 6m 55s	remaining: 9.44s
440:	learn 0.6721311294	total: 6m 56s	remaining: 8.49s
441:	learn 0.6721935741	total: 6m 56s	remaining: 7.54s
442:	learn 0.6722617747	total: 6m 57s	remaining: 6.6s
443:	learn 0.6723351475	total: 6m 58s	remaining: 5.65s
444:	learn 0.6724074442	total: 6m 58s	remaining: 4.71s
445:	learn 0.6724803401	total: 7m	remaining: 3.77s
446:	learn 0.6725049347	total: 7m 2s	remaining: 2.83s
447:	learn 0.6725573666	total: 7m 2s	remaining: 1.89s
448:	learn 0.6726688342	total: 7m 3s	remaining: 944ms
449:	learn 0.6727126071	total: 7m 4s	remaining: 0us


Скор 0.279

In [43]:
# Распарсим имена фич и все с суффиксом cat запишем в категориальные, получим их номера и будем 

def getCatFeatures(featureList):
    '''
    featureList is list of features name (df.columns.tolist())
    '''
    catFeatures = []
    for featureInd, featureName in enumerate(featureList):
        if featureName[-3:] == 'cat':
            catFeatures.append(featureInd)
            
    return catFeatures   

In [40]:
proba = np.ndarray([testDf.shape[0],5]) 
for seed in range(0,5):

    model = CatBoostClassifier(verbose= True, iterations= 550, thread_count= 4, eval_metric= "AUC",
                               depth= params['depth'], learning_rate= params['rate'],
                               l2_leaf_reg= params['l2'], bagging_temperature= params['T'],
                               random_seed= seed)
    model.fit(df.drop('target',axis=1), df.target)
    proba[:,seed] = model.predict_proba(testPool)[:,1]
    
submit(proba.mean(axis=1),'submission13.csv')    

Borders for float features generated
0:	learn 0.5000934936	total: 499ms	remaining: 4m 34s
1:	learn 0.5672959221	total: 1.27s	remaining: 5m 47s
2:	learn 0.584546325	total: 1.96s	remaining: 5m 57s
3:	learn 0.5940151299	total: 3.2s	remaining: 7m 17s
4:	learn 0.5971203595	total: 3.85s	remaining: 6m 59s
5:	learn 0.5999253437	total: 4.54s	remaining: 6m 52s
6:	learn 0.6065022303	total: 5.4s	remaining: 6m 59s
7:	learn 0.6056495869	total: 6.15s	remaining: 6m 56s
8:	learn 0.6069375993	total: 6.91s	remaining: 6m 55s
9:	learn 0.6095271382	total: 8.07s	remaining: 7m 16s
10:	learn 0.6102613597	total: 8.72s	remaining: 7m 7s
11:	learn 0.6124226506	total: 9.45s	remaining: 7m 3s
12:	learn 0.6155577105	total: 10.1s	remaining: 6m 56s
13:	learn 0.6181297131	total: 11.5s	remaining: 7m 21s
14:	learn 0.61826191	total: 12.2s	remaining: 7m 15s
15:	learn 0.6206948568	total: 12.9s	remaining: 7m 10s
16:	learn 0.6216294446	total: 13.7s	remaining: 7m 8s
17:	learn 0.6217101168	total: 14.4s	remaining: 7m 5s
18:	learn 

151:	learn 0.648751842	total: 2m 15s	remaining: 5m 56s
152:	learn 0.6488272353	total: 2m 16s	remaining: 5m 54s
153:	learn 0.6488827557	total: 2m 17s	remaining: 5m 53s
154:	learn 0.6489581806	total: 2m 18s	remaining: 5m 53s
155:	learn 0.6491606731	total: 2m 19s	remaining: 5m 52s
156:	learn 0.6493036862	total: 2m 20s	remaining: 5m 51s
157:	learn 0.6494229425	total: 2m 21s	remaining: 5m 49s
158:	learn 0.6495846083	total: 2m 21s	remaining: 5m 48s
159:	learn 0.6496682639	total: 2m 22s	remaining: 5m 47s
160:	learn 0.6497186627	total: 2m 23s	remaining: 5m 46s
161:	learn 0.6498475302	total: 2m 23s	remaining: 5m 44s
162:	learn 0.6499170963	total: 2m 24s	remaining: 5m 43s
163:	learn 0.6499319463	total: 2m 26s	remaining: 5m 44s
164:	learn 0.6500113033	total: 2m 27s	remaining: 5m 43s
165:	learn 0.6500970842	total: 2m 27s	remaining: 5m 42s
166:	learn 0.6502062172	total: 2m 28s	remaining: 5m 41s
167:	learn 0.6502467013	total: 2m 29s	remaining: 5m 40s
168:	learn 0.6503561722	total: 2m 30s	remaining: 

299:	learn 0.6616243297	total: 4m 41s	remaining: 3m 54s
300:	learn 0.6617248708	total: 4m 43s	remaining: 3m 54s
301:	learn 0.6617393932	total: 4m 45s	remaining: 3m 54s
302:	learn 0.6618200026	total: 4m 45s	remaining: 3m 53s
303:	learn 0.6618831566	total: 4m 46s	remaining: 3m 51s
304:	learn 0.6619073996	total: 4m 47s	remaining: 3m 50s
305:	learn 0.6619973847	total: 4m 49s	remaining: 3m 50s
306:	learn 0.6620851615	total: 4m 51s	remaining: 3m 50s
307:	learn 0.6621286883	total: 4m 52s	remaining: 3m 49s
308:	learn 0.6621784788	total: 4m 52s	remaining: 3m 48s
309:	learn 0.6622671898	total: 4m 54s	remaining: 3m 47s
310:	learn 0.662319393	total: 4m 55s	remaining: 3m 46s
311:	learn 0.662418818	total: 4m 55s	remaining: 3m 45s
312:	learn 0.6624875106	total: 4m 57s	remaining: 3m 45s
313:	learn 0.66254871	total: 4m 58s	remaining: 3m 44s
314:	learn 0.6627021706	total: 4m 59s	remaining: 3m 43s
315:	learn 0.6627965901	total: 5m 1s	remaining: 3m 43s
316:	learn 0.6628368303	total: 5m 2s	remaining: 3m 42

447:	learn 0.6710030935	total: 7m 10s	remaining: 1m 37s
448:	learn 0.6710237151	total: 7m 11s	remaining: 1m 37s
449:	learn 0.6711100189	total: 7m 13s	remaining: 1m 36s
450:	learn 0.6711434809	total: 7m 14s	remaining: 1m 35s
451:	learn 0.671234552	total: 7m 15s	remaining: 1m 34s
452:	learn 0.6713008983	total: 7m 16s	remaining: 1m 33s
453:	learn 0.6713328707	total: 7m 17s	remaining: 1m 32s
454:	learn 0.671371758	total: 7m 17s	remaining: 1m 31s
455:	learn 0.6714363056	total: 7m 19s	remaining: 1m 30s
456:	learn 0.6715165581	total: 7m 20s	remaining: 1m 29s
457:	learn 0.6715367936	total: 7m 20s	remaining: 1m 28s
458:	learn 0.6715952602	total: 7m 21s	remaining: 1m 27s
459:	learn 0.6716474586	total: 7m 22s	remaining: 1m 26s
460:	learn 0.6716937021	total: 7m 23s	remaining: 1m 25s
461:	learn 0.6717530271	total: 7m 24s	remaining: 1m 24s
462:	learn 0.671810012	total: 7m 24s	remaining: 1m 23s
463:	learn 0.6718435003	total: 7m 25s	remaining: 1m 22s
464:	learn 0.6718772919	total: 7m 26s	remaining: 1m

47:	learn 0.632763899	total: 42.7s	remaining: 7m 26s
48:	learn 0.6328871381	total: 43.5s	remaining: 7m 24s
49:	learn 0.633284006	total: 44.2s	remaining: 7m 22s
50:	learn 0.6337813806	total: 45.4s	remaining: 7m 24s
51:	learn 0.6345411397	total: 46.2s	remaining: 7m 22s
52:	learn 0.634400131	total: 46.9s	remaining: 7m 19s
53:	learn 0.6349320469	total: 47.6s	remaining: 7m 17s
54:	learn 0.6354301896	total: 48.3s	remaining: 7m 14s
55:	learn 0.6354487173	total: 49.1s	remaining: 7m 12s
56:	learn 0.6356839314	total: 49.8s	remaining: 7m 10s
57:	learn 0.635812651	total: 50.9s	remaining: 7m 12s
58:	learn 0.636014589	total: 51.7s	remaining: 7m 10s
59:	learn 0.636047151	total: 52.4s	remaining: 7m 8s
60:	learn 0.6361479812	total: 53.2s	remaining: 7m 6s
61:	learn 0.6363588476	total: 54.1s	remaining: 7m 6s
62:	learn 0.6365020158	total: 55.6s	remaining: 7m 9s
63:	learn 0.6368576328	total: 57.3s	remaining: 7m 14s
64:	learn 0.6369321049	total: 58.4s	remaining: 7m 15s
65:	learn 0.6370780323	total: 59.2s	re

...

436:	learn 0.6711992156	total: 6m 48s	remaining: 1m 45s
437:	learn 0.671219803	total: 6m 49s	remaining: 1m 44s
438:	learn 0.6712727832	total: 6m 49s	remaining: 1m 43s
439:	learn 0.671363219	total: 6m 50s	remaining: 1m 42s
440:	learn 0.6714356952	total: 6m 51s	remaining: 1m 41s
441:	learn 0.6715042359	total: 6m 52s	remaining: 1m 40s
442:	learn 0.6715085518	total: 6m 54s	remaining: 1m 40s
443:	learn 0.6715488667	total: 6m 54s	remaining: 1m 39s
444:	learn 0.6715853464	total: 6m 55s	remaining: 1m 38s
445:	learn 0.6716323689	total: 6m 56s	remaining: 1m 37s
446:	learn 0.6716823602	total: 6m 57s	remaining: 1m 36s
447:	learn 0.6717218128	total: 6m 57s	remaining: 1m 35s
448:	learn 0.6717684577	total: 6m 59s	remaining: 1m 34s
449:	learn 0.6718109054	total: 6m 59s	remaining: 1m 33s
450:	learn 0.6718461725	total: 7m	remaining: 1m 32s
451:	learn 0.6718674475	total: 7m 1s	remaining: 1m 31s
452:	learn 0.671883096	total: 7m 1s	remaining: 1m 30s
453:	learn 0.671916027	total: 7m 2s	remaining: 1m 29s
454

36:	learn 0.6315572886	total: 34.4s	remaining: 7m 57s
37:	learn 0.6316631943	total: 36.2s	remaining: 8m 7s
38:	learn 0.6320168612	total: 37.9s	remaining: 8m 16s
39:	learn 0.6321526101	total: 38.6s	remaining: 8m 12s
40:	learn 0.6325636717	total: 39.3s	remaining: 8m 8s
41:	learn 0.6327617193	total: 40.1s	remaining: 8m 5s
42:	learn 0.6329237705	total: 40.8s	remaining: 8m 1s
43:	learn 0.6333234228	total: 41.6s	remaining: 7m 57s
44:	learn 0.6336231575	total: 42.3s	remaining: 7m 55s
45:	learn 0.6337576683	total: 43.4s	remaining: 7m 55s
46:	learn 0.6339525106	total: 44.1s	remaining: 7m 51s
47:	learn 0.6342449805	total: 44.8s	remaining: 7m 48s
48:	learn 0.6342909879	total: 45.5s	remaining: 7m 45s
49:	learn 0.634361989	total: 47.3s	remaining: 7m 52s
50:	learn 0.634329198	total: 48s	remaining: 7m 50s
51:	learn 0.6347124031	total: 48.8s	remaining: 7m 46s
52:	learn 0.6350362348	total: 49.5s	remaining: 7m 44s
53:	learn 0.6351842447	total: 50.2s	remaining: 7m 41s
54:	learn 0.6354034041	total: 51s	re

185:	learn 0.653842278	total: 2m 52s	remaining: 5m 37s
186:	learn 0.6539565534	total: 2m 54s	remaining: 5m 38s
187:	learn 0.6540534087	total: 2m 55s	remaining: 5m 37s
188:	learn 0.6541108971	total: 2m 55s	remaining: 5m 35s
189:	learn 0.6542711138	total: 2m 56s	remaining: 5m 34s
190:	learn 0.6543240176	total: 2m 57s	remaining: 5m 33s
191:	learn 0.6544466865	total: 2m 58s	remaining: 5m 33s
192:	learn 0.6544760253	total: 2m 59s	remaining: 5m 32s
193:	learn 0.65457039	total: 3m	remaining: 5m 30s
194:	learn 0.6546972008	total: 3m 1s	remaining: 5m 29s
195:	learn 0.6547645041	total: 3m 1s	remaining: 5m 28s
196:	learn 0.6548911122	total: 3m 2s	remaining: 5m 26s
197:	learn 0.655010859	total: 3m 3s	remaining: 5m 26s
198:	learn 0.6551182983	total: 3m 4s	remaining: 5m 25s
199:	learn 0.6551802545	total: 3m 5s	remaining: 5m 23s
200:	learn 0.6552786882	total: 3m 5s	remaining: 5m 22s
201:	learn 0.6553557374	total: 3m 6s	remaining: 5m 21s
202:	learn 0.6554872426	total: 3m 7s	remaining: 5m 20s
203:	lear

333:	learn 0.6651147305	total: 5m 10s	remaining: 3m 20s
334:	learn 0.6651603554	total: 5m 11s	remaining: 3m 19s
335:	learn 0.6652622262	total: 5m 11s	remaining: 3m 18s
336:	learn 0.66527217	total: 5m 12s	remaining: 3m 17s
337:	learn 0.665389557	total: 5m 13s	remaining: 3m 16s
338:	learn 0.6654371568	total: 5m 14s	remaining: 3m 15s
339:	learn 0.6654999174	total: 5m 15s	remaining: 3m 14s
340:	learn 0.6655250288	total: 5m 16s	remaining: 3m 13s
341:	learn 0.6656026102	total: 5m 16s	remaining: 3m 12s
342:	learn 0.6656696471	total: 5m 17s	remaining: 3m 11s
343:	learn 0.6657419066	total: 5m 18s	remaining: 3m 10s
344:	learn 0.665796701	total: 5m 18s	remaining: 3m 9s
345:	learn 0.6658564768	total: 5m 19s	remaining: 3m 8s
346:	learn 0.6659574142	total: 5m 20s	remaining: 3m 7s
347:	learn 0.6660061053	total: 5m 21s	remaining: 3m 6s
348:	learn 0.6661074678	total: 5m 21s	remaining: 3m 5s
349:	learn 0.6661342971	total: 5m 23s	remaining: 3m 4s
350:	learn 0.6661688606	total: 5m 24s	remaining: 3m 3s
351

481:	learn 0.6744259443	total: 7m 17s	remaining: 1m 1s
482:	learn 0.6744742615	total: 7m 18s	remaining: 1m
483:	learn 0.674483127	total: 7m 20s	remaining: 1m
484:	learn 0.6745661324	total: 7m 21s	remaining: 59.2s
485:	learn 0.6745865718	total: 7m 22s	remaining: 58.2s
486:	learn 0.6746184964	total: 7m 22s	remaining: 57.3s
487:	learn 0.6746699072	total: 7m 23s	remaining: 56.4s
488:	learn 0.6747659178	total: 7m 24s	remaining: 55.4s
489:	learn 0.6748861542	total: 7m 24s	remaining: 54.5s
490:	learn 0.6749216058	total: 7m 25s	remaining: 53.6s
491:	learn 0.6749957321	total: 7m 26s	remaining: 52.6s
492:	learn 0.675114924	total: 7m 27s	remaining: 51.7s
493:	learn 0.6751363951	total: 7m 27s	remaining: 50.8s
494:	learn 0.6751660008	total: 7m 28s	remaining: 49.9s
495:	learn 0.6752300368	total: 7m 29s	remaining: 48.9s
496:	learn 0.6752525142	total: 7m 30s	remaining: 48s
497:	learn 0.6753087215	total: 7m 30s	remaining: 47.1s
498:	learn 0.6753538069	total: 7m 31s	remaining: 46.2s
499:	learn 0.6753747

Скор 0.279

In [50]:
proba = np.ndarray([testDf.shape[0],5]) 
for seed in range(0,5):
    print 'Iteration #', seed
    model = CatBoostClassifier(verbose= False, iterations= 700, thread_count= 4, eval_metric= "AUC",
                               depth= params['depth'], learning_rate= params['rate'],
                               l2_leaf_reg= params['l2'], bagging_temperature= params['T'],
                               random_seed= seed)
    print 'Learn'
    model.fit( Pool(df.drop('target',axis=1), df.target,
                    feature_names= df.columns.tolist(),
                    cat_features= getCatFeatures(df.drop('target',axis=1).columns.tolist())
                   )
             )
    print 'Trees number is ', model.tree_count_
    proba[:,seed] = model.predict_proba(  Pool(testDf,
                                               feature_names=testDf.columns.tolist(),
                                               cat_features= getCatFeatures(testDf.columns.tolist())
                                              )
                                       )[:,1]
    
submit(proba.mean(axis=1),'submission15.csv')    

Iteration # 0
Learn
Trees number is  700
Iteration # 1
Learn
Trees number is  700
Iteration # 2
Learn
Trees number is  700
Iteration # 3
Learn
Trees number is  700
Iteration # 4
Learn
Trees number is  700
